In [1]:
#notes for the future:
#1 -- should externalize list of tournaments to load
#     use json
#2 -- fix the first-round 'only give fifty percent of the pool points' thing
#3 -- need 'movers' and 'new schools' reports in spring reports
#4 -- port this script to be command-line compatible. ideally two command line arguments: year and season.
#maybe these changes should be draft 3

In [14]:
### import statements
import pandas as pd
pd.options.mode.chained_assignment = None  #supress warnings, i've tested the code
import numpy as np
from enum import Enum
import os
import re
import docx
from docxcompose.composer import Composer

In [3]:
### global definitions
def points_from_prelims(prelim_percentage):
    points = 8
    points += prelim_percentage>0
    points += prelim_percentage>=0.21
    points += prelim_percentage>=0.33
    points += prelim_percentage>0.4999
    points += prelim_percentage>0.50
    points += prelim_percentage>=0.67
    points += prelim_percentage>=0.80
    points += prelim_percentage>0.9999
    return points

def winner_points_from_elims(loser_ballots):
    return 6-(loser_ballots>=1)
def loser_points_from_elims(loser_ballots):
    return 3+(loser_ballots>=1)

class Division(Enum):
    VARSITY = 'v'
    JUNIOR_VARSITY = 'jv'
    NOVICE = 'n'
    ROUND_ROBIN = 'rr'
    TOTAL = 'total' ##hack. TODO: Remove.

class tournament():
        def __init__(self,tournament_name,tournament_year,prelim_count_vector,division_vector):
            self.prelim_counts = prelim_count_vector
            self.divisions = division_vector
            self.name = tournament_name
            self.year = tournament_year

MINIMUM_SCHOOLS_PER_DIVISION = 3
MINIMUM_TEAMS_PER_DIVISION = 6
MINIMUM_PRELIMS_PER_DIVISION = 4
MAXIMUM_ENTRIES_COUNTED_PER_SCHOOL = 2
MAXIMUM_TOURNAMENTS_COUNTED_PER_SCHOOL = 8
YEAR_TO_PROCESS = 2023
REPORT_TO_GENERATE = 1

NDT_DISTRICTS = range(1,9,1)

def first_or_second():
    if REPORT_TO_GENERATE==1:
        ordinal="first"
        season="fall"
    else:
        ordinal="second"
        season="spring"
    return [ordinal,season]

In [4]:
### turn CSV files from tabroom into sweepstakes points. 
#may properly drop invalid divisions, will certainly at least error out if presented with an invalid division.
#does not properly consider prelim seed in first elim round
#assumes any empty 'neg' entries represent byes for the listed aff team 
#    i don't think i properly handle 'implied walkovers': 
#    'liberty XX, liberty YY, <blank>' probably credits liberty XX for the points independent of what *should* be done.
#    we do print an error for it and call for manual intervention.
#does not account for 'extenuating circumstances', II.(g), needs more code

def is_division_valid(prelim_record,prelim_count):
    school_count = prelim_record['School'].nunique()
    entry_count = len(prelim_record['Code'])
    return (school_count>=MINIMUM_SCHOOLS_PER_DIVISION) & (entry_count>=MINIMUM_TEAMS_PER_DIVISION) & (prelim_count>=MINIMUM_PRELIMS_PER_DIVISION)

def process_elim_walkovers(elim_record): # takes canonical '<AFF TEAM> advances', processes them, replaces rows and returns.
    elim_walkovers = pd.DataFrame()
    elim_walkovers = elim_record[elim_record['Win'].str.contains('advances')]
    if not elim_walkovers.empty:
        elim_walkovers['walkover_winner'] = elim_walkovers['Win'].str[:-9]
        elim_walkovers['aff_walks_over'] = elim_walkovers['walkover_winner'] == elim_walkovers['Aff']
        elim_walkovers['walkover_ballot'] = elim_walkovers['aff_walks_over'].replace({True: '3-0\tAFF', False: '3-0\tNEG'})
        elim_walkovers.drop(columns=['walkover_winner','aff_walks_over','Win'],axis=1,inplace=True)
        elim_walkovers['Win'] = elim_walkovers['walkover_ballot']
        elim_walkovers.drop('walkover_ballot',axis=1,inplace=True)
        elim_record.drop(elim_record.index[elim_record['Win'].str.contains('advances')],inplace=True)
        elim_record = pd.concat([elim_record,elim_walkovers[['Aff','Neg','Win']]])
    return elim_record

def detect_implied_walkovers(elim_record):
    elim_record['implied_walkover'] = (elim_record['Win'].isna()) & (~(elim_record['Neg'].isna())) & (~(elim_record['Aff'].isna()))
    return elim_record['implied_walkover'].any()

def eliminate_blank_teams(elim_record): # replaces NaN and blanks with acceptable input for bye and walkover processing
    elim_record
    na_values = {'Neg': 'BLANK_ENTRY', 'Win': 'Aff BYE'}
    elim_record.fillna(value=na_values,inplace=True)
    return elim_record

def process_elim_forfeits(elim_record):
    elim_forfeits = pd.DataFrame()
    elim_forfeits = elim_record[elim_record['Win'].str.contains('FFT')]
    if not elim_forfeits.empty:
        elim_forfeits[['aff_result','neg_result']] = elim_forfeits['Win'].str.split('\t+',expand=True)
        elim_forfeits['aff_result'] = elim_forfeits['aff_result'].str[4:]
        elim_forfeits['Win'] = elim_forfeits['aff_result'].replace({'FFT': '3-0\tNEG','BYE':'3-0\tAff'})
        elim_record = elim_record.drop(elim_record.index[elim_record['Win'].str.contains('FFT')])
        elim_record = pd.concat([elim_record,elim_forfeits])
    return elim_record

    

def process_elim_byes(elim_record): # awards a 3-0 to the team getting a bye
    elim_byes = pd.DataFrame()
    elim_byes = elim_record[elim_record['Win'].str.contains('BYE')]
    if not elim_byes.empty:
        elim_byes[['Win','bye']] = elim_byes['Win'].str.split(' ',expand=True)
        elim_byes['Win'] = elim_byes['Win'].str.upper()
        elim_byes[['bye']] = elim_byes[['bye']].replace('BYE','3-0\t')
        elim_byes['Win'] = elim_byes['bye'] + elim_byes['Win']
        elim_byes = elim_byes.drop('bye',axis=1)
        elim_record = elim_record.drop(elim_record.index[elim_record['Win'].str.contains('BYE')])
        elim_record = pd.concat([elim_record,elim_byes])
    return elim_record

def drop_hybrid_entries(tournament_points):
    tournament_points['is_team_hybrid'] = tournament_points['Code'].str.contains('/')
    tournament_points.drop(tournament_points[tournament_points.is_team_hybrid].index, inplace=True)
    tournament_points.drop(['is_team_hybrid'],axis=1,inplace=True)
    return tournament_points

def process_points_division(tournament_name,year,prelim_count,division):
    data_folder = 'tournament_results/'+str(year)+'/'+tournament_name
    prelimFilePath=data_folder+'/'+tournament_name+'-'+division.value+'-prelims.csv'
    tournament_prelims = pd.read_csv(prelimFilePath)
    school_division_points = pd.DataFrame()
    if is_division_valid(tournament_prelims,prelim_count):
        tournament_prelims['prelim_winrate'] = tournament_prelims['Wins']/prelim_count
        tournament_prelims['prelim_points'] = tournament_prelims['prelim_winrate'].apply(points_from_prelims)
        tournament_points=tournament_prelims[['Code','School','prelim_points']]
        dir_list = os.listdir(data_folder)
        search_string = '-'+division.value+'-elim'
        elims_to_process = filter(lambda x: re.search(search_string, x), dir_list)
        elim_index=0
        for elim_filename in list(elims_to_process):
            elim_index+=1
            points_column_header='elim_'+str(elim_index)+"_points"
            elim_record=pd.read_csv(data_folder+'/'+elim_filename)[['Aff','Neg','Win']]
            if detect_implied_walkovers(elim_record):
                print("Human intervention needed: Implied walkover in "+elim_filename)
            elim_record = eliminate_blank_teams(elim_record)
            elim_record = process_elim_walkovers(elim_record)
            elim_record = process_elim_forfeits(elim_record)
            elim_record = process_elim_byes(elim_record)
            elim_record[['ballots','Win']] = elim_record['Win'].str.split('\t+',expand=True)
            elim_record[['winner_ballots','loser_ballots']] = elim_record['ballots'].str.split('-',expand=True)
            elim_record[['loser_ballots']] = elim_record[['loser_ballots']].astype("int")
            elim_record[['winner_points']] = elim_record[['loser_ballots']].apply(winner_points_from_elims)
            elim_record[['loser_points']] = elim_record[['loser_ballots']].apply(loser_points_from_elims)
            elim_record['aff_win'] = elim_record['Win'].apply(lambda y: 1 if y=='AFF' else 0)
            elim_record['neg_win'] = 1-elim_record['aff_win']
            elim_record['aff_points'] = elim_record['winner_points']*elim_record['aff_win']+elim_record['loser_points']*elim_record['neg_win']
            elim_record['neg_points'] = elim_record['winner_points']*elim_record['neg_win']+elim_record['loser_points']*elim_record['aff_win']
            temp_aff = pd.DataFrame()
            temp_neg = pd.DataFrame()
            temp_aff[['Code',points_column_header]] = elim_record[['Aff','aff_points']]
            temp_neg[['Code',points_column_header]] = elim_record[['Neg','neg_points']]
            elim_points = pd.concat([temp_aff,temp_neg])
            tournament_points = tournament_points.merge(elim_points,'left','Code')
            tournament_points[[points_column_header]] = tournament_points[[points_column_header]].fillna(0).astype(int)
        tournament_points = drop_hybrid_entries(tournament_points)
        tournament_points['total_points'] = tournament_points.drop(['Code','School'],axis=1).sum(axis=1)
        school_division_points = tournament_points[['School','total_points']].groupby('School',as_index=False).agg({'total_points': {lambda z: z.nlargest(MAXIMUM_ENTRIES_COUNTED_PER_SCHOOL).sum()}})
        school_division_points.columns = list(map(''.join, school_division_points.columns.values))
        school_division_points[tournament_name+'_'+division.value+'_points'] = school_division_points['total_points<lambda>']
        school_division_points = school_division_points.drop('total_points<lambda>',axis=1)
    return school_division_points
#temp1 = process_points_division("nuso","NUSO",2023,6,Division.VARSITY)
#print(temp1.to_string())
#temp2 = process_points_division("nuso","NUSO",2023,6,Division.JUNIOR_VARSITY)
#print(temp2.to_string())

In [5]:
### Functions to split tournaments into divisions, and integrate tournaments into one Big Table
def process_points_tournament(tournament):
    tournament_name=tournament.name
    prelim_count_vector=tournament.prelim_counts
    division_vector=tournament.divisions
    year=tournament.year
    school_tournament_points=pd.DataFrame()
    for (division,prelim_count) in zip(division_vector, prelim_count_vector):
        division_points = pd.DataFrame()
        division_points = process_points_division(tournament_name,year,prelim_count,division)
        if school_tournament_points.empty:
            school_tournament_points = division_points
        else:
            school_tournament_points = school_tournament_points.merge(division_points,how='outer',on='School')
    school_tournament_points.fillna(0,inplace=True)
    columns_to_add = school_tournament_points.loc[:,school_tournament_points.columns!='School']
    total_tournament_points = columns_to_add.sum(axis=1)
    school_tournament_points[tournament_name+'_total_points'] = total_tournament_points
    return school_tournament_points

def tournament_merge(cumulative_list,new_tournament):
    if cumulative_list.empty:
        return new_tournament
    new_cumulative_list = cumulative_list.merge(new_tournament,how='outer',on='School')
    new_cumulative_list.fillna(0,inplace=True)
    return new_cumulative_list

def sum_legal_tournaments(cumulative_points,division,legal_tournament_count):
    column_label_substring = '_'+division.value+'_'
    filtered_cumulative_points = cumulative_points.filter(like=column_label_substring)
    filtered_cumulative_points = filtered_cumulative_points.apply(pd.Series.nlargest,axis=1,n=legal_tournament_count)
    filtered_cumulative_points.fillna(0,inplace=True)
    total_points=pd.DataFrame()
    total_points['School'] = cumulative_points['School']
    total_points[division.value+'_total_points'] = filtered_cumulative_points.sum(axis=1)
    return total_points

#temp3 = process_points_tournament(tournament_george_mason)
#print(temp3.to_string())
#temp4 = process_points_tournament(tournament_northwestern)
#print(temp4.to_string())
#print(temp3.merge(temp4,how='outer',on='School'))

In [6]:
### define tournaments and execute
tournament_northwestern = tournament('nuso',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.JUNIOR_VARSITY])
tournament_george_mason = tournament('gmu',YEAR_TO_PROCESS,[6,6,5],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_mostate = tournament('mostate',YEAR_TO_PROCESS,[6,5,6],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_binghamton = tournament('binghamton',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.NOVICE])
tournament_kentucky = tournament('kentucky',YEAR_TO_PROCESS,[6,6,6,6],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE,Division.ROUND_ROBIN])
tournament_houston = tournament('houston',YEAR_TO_PROCESS,[6,6,6],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_mukai = tournament('mukai',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.JUNIOR_VARSITY])
tournament_westpoint = tournament('westpoint',YEAR_TO_PROCESS,[6,6,6],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_wayne = tournament('wayne',YEAR_TO_PROCESS,[6,5],[Division.VARSITY,Division.JUNIOR_VARSITY])
tournament_harvard = tournament('harvard',YEAR_TO_PROCESS,[8,6,6],[Division.VARSITY,Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_uco = tournament('uco',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.NOVICE])
tournament_wake = tournament('wake',YEAR_TO_PROCESS,[7],[Division.VARSITY])
tournament_adafc = tournament('adafc',YEAR_TO_PROCESS,[6,6],[Division.JUNIOR_VARSITY,Division.NOVICE])
tournament_monmouth = tournament('monmouth',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.NOVICE])
tournament_bto = tournament('bto',YEAR_TO_PROCESS,[6,6],[Division.VARSITY,Division.NOVICE])

cumulative_points = pd.DataFrame()
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_northwestern))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_george_mason))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_mostate))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_binghamton))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_kentucky))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_houston))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_mukai))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_westpoint))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_wayne))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_harvard))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_uco))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_wake))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_adafc))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_monmouth))
cumulative_points = tournament_merge(cumulative_points,process_points_tournament(tournament_bto))
#print(cumulative_points.to_string())

                      School  nuso_v_points  nuso_jv_points  nuso_total_points  gmu_v_points  gmu_jv_points  gmu_n_points  gmu_total_points  mostate_v_points  mostate_jv_points  mostate_n_points  mostate_total_points  binghamton_v_points  binghamton_n_points  binghamton_total_points  kentucky_v_points  kentucky_jv_points  kentucky_n_points  kentucky_rr_points  kentucky_total_points  houston_v_points  houston_jv_points  houston_n_points  houston_total_points  mukai_v_points  mukai_jv_points  mukai_total_points  westpoint_v_points  westpoint_jv_points  westpoint_n_points  westpoint_total_points  wayne_v_points  wayne_jv_points  wayne_total_points  harvard_v_points  harvard_jv_points  harvard_n_points  harvard_total_points  uco_v_points  uco_n_points  uco_total_points  wake_v_points  wake_total_points  adafc_jv_points  adafc_n_points  adafc_total_points  monmouth_v_points  monmouth_n_points  monmouth_total_points  bto_v_points  bto_n_points  bto_total_points
0                     Baylor  

In [7]:
### report generation
# need: new schools and movers: spring-only
#print(cumulative_points[cumulative_points['School'].str.contains('Suffolk')].transpose().to_string())
def add_rank_column(dataframe):
    dataframe['Rank'] = range(1,len(dataframe)+1)
    dataframe['Rank'] = dataframe['Rank'].astype(str)+'.'
    columns = dataframe.columns.tolist()
    columns = columns[-1:] + columns[:-1]
    return dataframe[columns]

temp5 = sum_legal_tournaments(cumulative_points,Division.TOTAL,MAXIMUM_TOURNAMENTS_COUNTED_PER_SCHOOL)
temp6 = sum_legal_tournaments(cumulative_points,Division.VARSITY,MAXIMUM_TOURNAMENTS_COUNTED_PER_SCHOOL)
sweepstakes_results_for_reports = pd.DataFrame()
sweepstakes_results_for_reports = temp5.merge(temp6,how='outer',on='School').fillna(0)
sweepstakes_results_for_reports['NDT pts'] = sweepstakes_results_for_reports.total_total_points.astype(int)
sweepstakes_results_for_reports['Varsity pts'] = sweepstakes_results_for_reports.v_total_points.astype(int)
sweepstakes_results_for_reports.drop(columns=['v_total_points','total_total_points'],inplace=True)
schools_by_districts = pd.read_csv('ndt-districts-'+str(YEAR_TO_PROCESS)+'.csv',quotechar="'")
community_colleges = pd.read_csv('community-colleges-'+str(YEAR_TO_PROCESS)+'.csv')
sweepstakes_results_for_reports = sweepstakes_results_for_reports.merge(schools_by_districts,how='left',on='School')
sweepstakes_results_for_reports = sweepstakes_results_for_reports.merge(community_colleges,how='left',on='School')
sweepstakes_results_for_reports.fillna(value=False,inplace=True)
sweepstakes_results_for_reports['CC'].replace({True: 'Y', False: 'N'},inplace=True)
#print(sweepstakes_results_for_reports.to_string())

sweepstakes_results_for_reports.to_csv(index=False,path_or_buf="sweepstakes_output_"+str(YEAR_TO_PROCESS)+"_full.csv")

sweepstakes_top10_overall = add_rank_column(sweepstakes_results_for_reports.sort_values('NDT pts',ascending=False,ignore_index=True).head(10))
sweepstakes_top10_varsity = add_rank_column(sweepstakes_results_for_reports.sort_values('Varsity pts',ascending=False,ignore_index=True).head(10))
sweepstakes_top10_overall_CC = add_rank_column(sweepstakes_results_for_reports[sweepstakes_results_for_reports['CC']=='Y'].sort_values('NDT pts',ascending=False,ignore_index=True))
sweepstakes_overall_rankings = add_rank_column(sweepstakes_results_for_reports.sort_values('NDT pts',ascending=False,ignore_index=True))
sweepstakes_varsity_rankings = add_rank_column(sweepstakes_results_for_reports.sort_values('Varsity pts',ascending=False,ignore_index=True))

#print(sweepstakes_top10_overall.to_string())
#print(sweepstakes_top10_varsity.to_string())
#print(sweepstakes_top10_overall_CC.to_string())


district_overall_sweepstakes_points = {}
for district in NDT_DISTRICTS:
    district_overall_sweepstakes_points[district] = add_rank_column(sweepstakes_results_for_reports[sweepstakes_results_for_reports['District']==district].sort_values('NDT pts',ascending=False,ignore_index=True))
#    print(district_overall_sweepstakes_points[district])

                      School  NDT pts  Varsity pts  District CC
0                     Baylor      161          111         3  N
1                 Binghamton      322          184         8  N
2             CSU Long Beach      157          135         1  N
3        Cal State Fullerton      144           72         1  N
4                  Dartmouth      157          145         8  N
5                      Emory      438          308         6  N
6               George Mason      604          171         7  N
7                 Georgetown      197          184         7  N
8                    Georgia      109           55         6  N
9                    Gonzaga      138          123         2  N
10                   Harvard      219          219         8  N
11                   Houston      325          148         3  N
12                   Indiana      221           62         5  N
13                      Iowa      184          176         4  N
14                    Kansas      299   

In [16]:
##output to word tables



[report_ordinal,report_season] = first_or_second()
season_caps=report_season.upper()
season_sentence=report_season.capitalize()
report_replacement_dictionary={"$YEAR":str(YEAR_TO_PROCESS),"$FIRST":report_ordinal,"$SEASON_LOWER":report_season,"$SEASON_UPPER":season_caps,"$SEASON_SENTENCE":season_sentence}


def report_update_year(template_document):
    for paragraph in template_document.paragraphs:
        for replacement_item in report_replacement_dictionary:
            if paragraph.text.find(replacement_item)>=0:
                runs=paragraph.runs
                for i in range(len(runs)):
                    if runs[i].text.find(replacement_item)>=0:
                        runs[i].text = runs[i].text.replace(replacement_item,report_replacement_dictionary[replacement_item])
    for replacement_item in report_replacement_dictionary:
        runs=template_document.sections[0].footer.paragraphs[0].runs
        for i in range(len(runs)):
            if runs[i].text.find(replacement_item)>=0:
                runs[i].text = runs[i].text.replace(replacement_item,report_replacement_dictionary[replacement_item])
                print(runs[i].text)
        for table in template_document.tables:
            for col in table.columns:
                for cell in col.cells:
                    for paragraph in cell.paragraphs:
                        if paragraph.text.find(replacement_item)>=0:
                            runs=paragraph.runs
                            for i in range(len(runs)):
                                if runs[i].text.find(replacement_item)>=0:
                                    runs[i].text = runs[i].text.replace(replacement_item,report_replacement_dictionary[replacement_item])
    return template_document
        

def append_word_results_table(results_document,results_dataframe,append_footers):
    results_column_widths=[0.5,2,0.7,0.9,0.7,0.4]
    created_table = results_document.add_table(results_dataframe.shape[0]+1,results_dataframe.shape[1],style="NDTSweepstakes")

    for j in range(results_dataframe.shape[-1]):
        created_table.cell(0,j).text = results_dataframe.columns[j]
        created_table.cell(0,j).width=docx.shared.Inches(results_column_widths[j])
    
    for i in range(results_dataframe.shape[0]):
        for j in range(results_dataframe.shape[-1]):
            created_table.cell(i+1,j).text=str(results_dataframe.values[i,j])
            created_table.cell(i+1,j).width=docx.shared.Inches(results_column_widths[j])
    
#    for cell in created_table.columns[0].cells:
#        cell.width=docx.shared.Inches(0.5)
#    for cell in created_table.columns[1].cells:
#        cell.width=docx.shared.Inches(2)
#    for cell in created_table.columns[2].cells:
#        cell.width=docx.shared.Inches(0.7)
#    for cell in created_table.columns[3].cells:
#        cell.width=docx.shared.Inches(0.9)
#    for cell in created_table.columns[4].cells:
#        cell.width=docx.shared.Inches(0.7)
#    for cell in created_table.columns[5].cells:
#        cell.width=docx.shared.Inches(0.4)
    if append_footers:
        results_document.add_paragraph('')
        footer_table=results_document.add_table(1,1,style="NDTSweepstakes")
        results_document.add_paragraph('')
    return results_document

def append_table_header(results_document,title_string):
    results_document.add_heading(title_string,level=3)
    results_document.add_paragraph('')
    return results_document

results_document = docx.Document('sweepstakes-table-template.docx')
results_document = report_update_year(results_document)

results_document = append_table_header(results_document,"Top 10 Overall Rankings")
results_document = append_word_results_table(results_document,sweepstakes_top10_overall,True)

results_document = append_table_header(results_document,"Top 10 Varsity Rankings")
results_document = append_word_results_table(results_document,sweepstakes_top10_varsity,True)

results_document = append_table_header(results_document,"Top CC Rankings")
results_document = append_word_results_table(results_document,sweepstakes_top10_overall_CC,True)
results_document.add_page_break()

results_document = append_table_header(results_document,"Overall Rankings")
results_document = append_word_results_table(results_document,sweepstakes_overall_rankings,True)
results_document.add_page_break()

results_document = append_table_header(results_document,"Varsity Rankings")
results_document = append_word_results_table(results_document,sweepstakes_varsity_rankings,True)
results_document.add_page_break()

results_document = append_table_header(results_document,"Overall Rankings by District")
for district in NDT_DISTRICTS:
    results_document = append_word_results_table(results_document,district_overall_sweepstakes_points[district],False)
    results_document.add_paragraph('')
footer_table=results_document.add_table(1,1,style="NDTSweepstakes")
results_document.add_paragraph('')
results_document.add_page_break()

results_composer=Composer(results_document)
procedure_document=docx.Document('sweepstakes-procedure.docx')

results_composer.append(procedure_document)

results_composer.save('test_document.docx')#'NDT-sweepstakes_tables_'+str(YEAR_TO_PROCESS))

$SEASON_SENTENCE 2023 NDT Ranking Report
2023
Fall 2023 NDT Ranking Report
Fall
Fall
